# JAMSTEC DB スキーム確認ツール for APEX(APF11)



In [170]:
import os
import pandas as pd
import re
import termcolor
import Levenshtein # レーベンシュタイン距離ライブラリにある、ジャロ・ウインクラー距離を計算するのに使う
# jaro_dist = Levenshtein.jaro_winkler(srt1 , str2)

apf11_excel = pd.read_excel('Apex_apf11.xlsx' , sheet_name=None) # sheet_name=Noneで全てのシート読み込む

### ジャロ・ウィンクラー距離法の関数

2つの文字列を移動したり修正したりした回数等で計算する。１で一致とみなされる。

Winkler, W. E. (1990). "String Comparator Metrics and Enhanced Decision Rules in the Fellegi-Sunter Model of Record Linkage". Proceedings of the Section on Survey Research Methods. American Statistical Association: 354–359.


In [171]:
def jaro_dist(str1,str2):
    return Levenshtein.jaro_winkler(str1,str2)

In [172]:
print(apf11_excel['scheme'].columns)

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')


### エクセルファイルの列を数えて抜き出す

In [173]:
msgcol = apf11_excel['scheme'].iloc[:,1]
a26msg = apf11_excel['scheme'].iloc[:,7] # ７列目がA26のmsg , 8列目はlog 以降同様に数える
x1a27system = apf11_excel['scheme'].iloc[:,10]
x2x3a28a29system = apf11_excel['scheme'].iloc[:,12]
a30a31x4system = apf11_excel['scheme'].iloc[:,14]
a33a34a35system = apf11_excel['scheme'].iloc[:,16]

scheme = pd.concat([msgcol,a26msg,x1a27system,x2x3a28a29system,a30a31x4system,a33a34a35system], axis=1)\
.rename(columns={'Unnamed: 1':'field_name', 'Unnamed: 7':'a26msg' , 'Unnamed: 10':'x1a27system' , 'Unnamed: 12':'x2x3a28a29system' , 'Unnamed: 14':'a30a31x4system' , 'Unnamed: 16':'a33a34a35system'})

techcol = apf11_excel['tech'].iloc[:,1]
a26msg = apf11_excel['tech'].iloc[:,7]
a26log = apf11_excel['tech'].iloc[:,8]
x1a27science = apf11_excel['tech'].iloc[:,9]
x1a27vitals = apf11_excel['tech'].iloc[:,10]
x1a27systemtech = apf11_excel['tech'].iloc[:,11]
x2x3a28a29a30a31a33a34science = apf11_excel['tech'].iloc[:,12]
x2x3a28a29a30a31a33a34vitals = apf11_excel['tech'].iloc[:,13]
x2x3a28a29a30a31a33a34systemtech = apf11_excel['tech'].iloc[:,14]
a28science = apf11_excel['tech'].iloc[:,15]
a28vitals = apf11_excel['tech'].iloc[:,16]
a28system = apf11_excel['tech'].iloc[:,17]

tech = pd.concat([techcol,a26msg,a26log,x1a27science,x1a27vitals,x1a27systemtech,x2x3a28a29a30a31a33a34science,x2x3a28a29a30a31a33a34vitals,x2x3a28a29a30a31a33a34systemtech,a28science,a28vitals,a28system], axis=1)\
.rename(columns={'Unnamed: 1':'field_name'\
                 , 'Unnamed: 7':'a26msg' , 'Unnamed: 8':'a26log'\
                 , 'Unnamed: 9':'x1a27sicence' , 'Unnamed: 10':'x1a27vitals' , 'Unnamed: 11':'x1a27system'\
                 , 'Unnamed: 12':'x2x3a28a29a30a31a33a34science' , 'Unnamed: 13':'x2x3a28a29a30a31a33a34vitals' , 'Unnamed: 14':'x2x3a28a29a30a31a33a34system'\
                 , 'Unnamed: 15':'a28science' , 'Unnamed: 16':'a28vitals' , 'Unnamed: 17':'a28system'})



### 生データ読み込み


In [174]:
with open('rawdata/A34/f8900.001.20200310T174432.system_log.txt','r') as sys:
    sysline = sys.readlines()
    
with open('rawdata/A34/f8900.001.20200310T174432.vitals_log.csv','r') as vit:
    vitline = vit.readlines()
    
with open('rawdata/A34/f8900.001.merge.science_log.csv','r') as sci:
    sciline = sci.readlines()


### system_log読み込み
各行末をパラメータ名と判断する

後ろからみて｜があったらそこまでをパラメータと判断する


In [175]:
regex = r'|$'
pt = re.compile(regex)

reg2 = r'^[A-Z]'
pt2 = re.compile(reg2)

sysdata = {}
i = 0
for line in sysline:
    if(pt.match(line)):
        param = str(line.split('|')[-1:])
        # 前と後ろの邪魔な文字を削除
        para = param[2:]
        para2 = para[:-4]
        # 数字は削除する,先頭のみ取得
        para3 = str(para2.split(' ')[0])
        if(pt2.match(para3)):
            sysdata[i] = para3
            i += 1

#print(sysdata)
sysdf = pd.DataFrame(sysdata.values() , columns={'field_name'})
#print(sysdf)

### science_log 読み込み
CTD_P/CTD_PTS/CTD_CPと数値があるのみなので比較する必要なし


### vitals_log 読み込み

vitals_coreがカンマ区切りなのでそのまま読み取る。
カラム数が重要（５項目めがBatteryVoltage等と決まる

###### Excel の内容判断は文章化されていて困難なのでテーブルとして以下を予め設定する（２０２０１０最新のA34が基準）
|Column|DB table name|
| ---- | ---- |
|3|AirBladderPressure|
|5|voltage_load<br>QuiescentVolts<br>Sbe41cpVolts<br>BuoyancyPumpVolts<br>BuoyancyPumpVolts2<br>AirPumpVolts|
|7|Humidity|
|8|LeakDetectVolts|
|9|Vacuum|
|11|CoulmbCounterAmphrs|


In [176]:
regex = r'^VITALS' # VITALS_COREの行は取得する
pattern = re.compile(regex)
vitdata = []

for line in vitline:
    if(pattern.match(line)):
        vitdata.append(line.split(','))
        print(line.split(','))
        

['VITALS_CORE', '20200308T014341', '11.709', '2352', '15.311', '3484', '9.859', '2.324', '7.410', '2914', '2175.462', '53.910', '-690\n']
['VITALS_CORE', '20200308T014345', '11.765', '2363', '15.302', '3482', '9.830', '2.324', '7.420', '2907', '2175.462', '35.393', '-453\n']
['VITALS_CORE', '20200308T024715', '7.697', '1573', '15.420', '3509', '10.993', '2.319', '8.006', '3138', '2217.097', '31.799', '-407\n']
['VITALS_CORE', '20200310T072607', '7.300', '1494', '15.372', '3498', '10.440', '2.303', '7.588', '2986', '2239.063', '30.861', '-395\n']
['VITALS_CORE', '20200310T104227', '7.285', '1491', '15.328', '3489', '10.382', '2.301', '7.583', '2978', '2249.657', '31.408', '-402\n']
['VITALS_CORE', '20200310T173723', '7.652', '1562', '15.021', '3417', '9.161', '2.321', '7.947', '3122', '2520.428', '31.174', '-399\n']
['VITALS_CORE', '20200310T174429', '11.938', '2402', '14.915', '3393', '8.579', '2.304', '7.122', '2797', '2533.866', '52.269', '-669\n']


In [177]:
# 以下のような感じで数値が入ってたら項目ありと判断するだけ。
print((vitdata[0][4]))

15.311


In [178]:
print(scheme['a33a34a35system'])

0                     NaN
1              system_log
2            ParkPressure
3     DeepDescentPressure
4         DownTime+UpTime
             ...         
61                    NaN
62                    NaN
63                    NaN
64                    NaN
65                    NaN
Name: a33a34a35system, Length: 66, dtype: object


In [179]:
print(sysdf['field_name'][37:])

37       PreludeSelfTest
38           PreludeTime
39       SurfacePressure
40         TelemetryDays
41     TelemetryInterval
             ...        
466                  Set
467                  GPS
468                  GPS
469                  GPS
470                 Time
Name: field_name, Length: 434, dtype: object


In [188]:
# 結果保存用のDFを準備
resultdf = pd.DataFrame(columns=['index','msg','xls','score'])

for line in sysdf['field_name']:
    query = scheme['a33a34a35system'].str.startswith(line , na=False)
    #print(query.values)
    if (query[query == True].first_valid_index()):
        print( termcolor.colored(line + ' field exists.','blue'))
    else:
        print( termcolor.colored(line + ' is not found.' , 'red'))
        
        for index,item in scheme.iterrows():
            score = jaro_dist(line , str(item['a33a34a35system']) ) # 引数を入れ替えると結果が多少変わる。
            #print(line + ' is probably ' + str(item['a33a34a35system']) + ' ( ' + str(round(score,2)*100) + '%)' )
            record = pd.Series([index , line , item['a33a34a35system'] , score] , index=resultdf.columns)
            resultdf = resultdf.append(record , ignore_index=True)


Archived: is not found.
Archived: is not found.
Archived: is not found.
PRELUDE is not found.
Starting is not found.
ActivateRecoveryMode is not found.
AscentRate field exists.
AscentStartTimes field exists.
AscentTimeout field exists.
AscentTimerInterval is not found.
BuoyancyNudge field exists.
DeepDescentCount field exists.
DeepDescentPressure field exists.
DeepDescentTimeout field exists.
DeepDescentTimerInterval is not found.
DeepProfileFirst field exists.
DownTime field exists.
EmergencyTimerInterval is not found.
HyperRetractCount field exists.
HyperRetractPressure field exists.
IceMonths field exists.
IdleTimerInterval field exists.
InitialBuoyancyNudge field exists.
LeakDetect is not found.
LogVerbosity field exists.
MActivationCount is not found.
MActivationPressure is not found.
MinBuoyancyCount field exists.
MinVacuum is not found.
ParkBuoyancyNudge field exists.
ParkDeadBand field exists.
ParkDescentCount field exists.
ParkDescentTimeout field exists.
ParkDescentTimerInter

In [189]:
print(resultdf)

     index        msg                  xls     score
0        0  Archived:                  NaN  0.000000
1        1  Archived:           system_log  0.403704
2        2  Archived:         ParkPressure  0.462963
3        3  Archived:  DeepDescentPressure  0.275828
4        4  Archived:      DownTime+UpTime  0.451852
...    ...        ...                  ...       ...
6925    61       Time                  NaN  0.000000
6926    62       Time                  NaN  0.000000
6927    63       Time                  NaN  0.000000
6928    64       Time                  NaN  0.000000
6929    65       Time                  NaN  0.000000

[6930 rows x 4 columns]


### ソートして上位３件を表示

In [190]:
items = len(scheme.dropna()) # Index数、この数分ジャロ・ウインクラー距離を計算したら次のマッチしなかった語句になる
kazu = int( len(resultdf) / items ) # クエリーのリストになくnot foundで表示した数

for count in range(kazu):
    res = resultdf[items*count : items*(count+1)]
    msgrank = res.sort_values('score',ascending=False)[:3] # Score降順にソートして上から3つを表示

    if((re.match('^.',msgrank.iat[0,1])) is not None  ):
        # print(ranking.iat[0,3])
        # score が０と１の時は抜く（アルゴリズム？で１００％が結構出てる。
        disp_rank = str(msgrank.iat[0,1]) + ' is probably ' + str(msgrank.iat[0,2]) + ' ( ' + str(round(msgrank.iat[0,3] , 2) * 100 ) + '% )' + '\n' \
        + ' or ' + str(msgrank.iat[1,2]) + ' ( ' + str(round(msgrank.iat[1,3] , 2) * 100 ) + '% )' + '\n' \
        + ' or ' + str(msgrank.iat[2,2]) + ' ( ' + str(round(msgrank.iat[2,3] , 2) * 100 ) + '% )' + '\n' 
    
        print( disp_rank )

Archived: is probably UpTime ( 52.0% )
 or SurfacePressure ( 51.0% )
 or DownTime ( 49.0% )

Archived: is probably AscentStartTimes ( 56.00000000000001% )
 or AscentStartTimes ( 56.00000000000001% )
 or AscentStartTimes ( 56.00000000000001% )

Archived: is probably AscentRate ( 59.0% )
 or AscentTimeout ( 55.00000000000001% )
 or PnPCycleLen ( 47.0% )

Archived: is probably IceMonths ( 44.0% )
 or nan ( 0.0% )
 or nan ( 0.0% )

Archived: is probably SurfacePressure ( 51.0% )
 or ParkPressure ( 46.0% )
 or DownTime+UpTime ( 45.0% )

Archived: is probably AscentStartTimes ( 56.00000000000001% )
 or AscentStartTimes ( 56.00000000000001% )
 or AscentStartTimes ( 56.00000000000001% )

Archived: is probably AscentRate ( 59.0% )
 or AscentTimeout ( 55.00000000000001% )
 or ParkDeadBand ( 53.0% )

Archived: is probably IceMonths ( 44.0% )
 or TelemetryRetryInterval ( 39.0% )
 or nan ( 0.0% )

Archived: is probably SurfacePressure ( 51.0% )
 or ParkPressure ( 46.0% )
 or DownTime+UpTime ( 45.0%